# Descripción del sistema
El sistema consiste en el desarrollo de 5 gráficos diferentes, estos gráficos son realizados con dos conjuntos de datos de dos entidades costarricenses diferentes que son el INEC y el OIJ, ambos conjuntos de datos fueron descargados en formato csv y son almacenados como dataframes utilizando la función de spark en python, además de estos los datos son limpiados ya que tienen incongruencias en su formato, una vez con los datos limpios se envían los mismo a postgres y se grafican

In [2]:
#Bibliotecas necesarias para el uso de spark
from pyspark.sql import SparkSession
from pyspark.files import SparkFiles
from pyspark.sql import functions as F
from pyspark.sql.functions import col, unix_timestamp, to_date , regexp_replace
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
#Se guardan ambos archivos como dataframes
oij_csv=spark.read.format("csv").option("header","true").option("inferSchema","true").load("Estadisticas.csv")
inec_csv=spark.read.format("csv").option("header","true").option("inferSchema","true").load("reempleocenso2011-22.csv")

In [ ]:
#Se imprimen ambos esquemas para observar el formato en el que vienen los datos, esto servirá para ajustar
#el dataframe a lo que necesitemos
#este de abajo será el formato del OIJ
oij_csv.printSchema()
oij_csv.show()

In [ ]:
#Este otro formato correspone al INEC
inec_csv=inec_csv.withColumn('PoblacionMayor15', regexp_replace(col('PoblacionMayor15'), " ", ""))
inec_csv=inec_csv.withColumn('PoblacionMayor15',inec_csv['PoblacionMayor15'].cast("Integer").alias('PoblacionMayor15'))
inec_csv=inec_csv.withColumn('TasaDesempleoAbierto',inec_csv['TasaDesempleoAbierto'].cast("double").alias('TasaDesempleoAbierto'))
inec_csv.printSchema()

In [ ]:
#Se registran ambos dataframe como tablas de SQL, esto nos permite realizar consultas sql por medio de sqlContext

sqlContext.registerDataFrameAsTable(oij_csv, "oij")
sqlContext.registerDataFrameAsTable(inec_csv, "inec_csv")
#Se arregla el archivo del inec original para poder juntarlo correctamente
inec_csv1= sqlContext.sql("SELECT ProvinciaCantonDistrito, SUM(PoblacionMayor15) PoblacionMayor15, AVG(TasaNetaParticipacion)TasaNetaParticipacion,AVG(TasaOcupacion)TasaOcupacion,AVG(TasaDesempleoAbierto)TasaDesempleoAbierto,AVG(PorcentajeEconomicamenteInactivo)PorcentajeEconomicamenteInactivo, AVG(RelacionDependenciaEconomica)RelacionDependenciaEconomica FROM inec_csv GROUP BY ProvinciaCantonDistrito ")
sqlContext.registerDataFrameAsTable(inec_csv1, "inec")

In [ ]:
'''
Entradas: Un entero
Salidas: Un dataframe
Descripción general: Se encarga de validar cual conjunto de datos se debe enviar 
y realiza la consulta SQL correspondiente la cual removerá espacios en blanco
'''
def quitaEspacios(ind):
    #Este if es de validación para enviar datos de INEC o del OIJ
    #por medio de sqlContext se utiliza la función TRIM que acorta todos los espacios en blanco
    #almacenados antes o después de nuestros datos
    if ind == 1:
        return sqlContext.sql("SELECT Delito, SubDelito,Hora, Fecha, Victima, SubVictima, Edad, Genero, Nacionalidad, Provincia,Canton, TRIM(Distrito) as Distrito from oij")
    else:
        return sqlContext.sql("SELECT TRIM(ProvinciaCantonDistrito) AS ProvinciaCantonDistrito, PoblacionMayor15, TasaNetaParticipacion, TasaOcupacion, TasaDesempleoAbierto, PorcentajeEconomicamenteInactivo, RelacionDependenciaEconomica from inec")

In [ ]:
#Simplemente se llama a la función con el número entero correspondiente y se registra el nuevo dataframe
#como la tabla almacenada anteriormente en el sistema, esto se aplica para ambos conjuntos de datos
oij = quitaEspacios(1)
sqlContext.registerDataFrameAsTable(oij, "oij")
inec = quitaEspacios(2)
sqlContext.registerDataFrameAsTable(inec, "inec")

In [ ]:
'''
Entradas: Un entero
Salidas: Un dataframe
Descripción general: Se encarga de validar cual conjunto de datos se debe enviar 
y realiza la consulta SQL correspondiente la cual pasará los datos a minúscula
'''
def minusculas(ind):
    #Este if es de validación para enviar datos de INEC o del OIJ
    #por medio de sqlContext se utiliza la función LOWER que se encarga de pasar los datos a minúscula
    if ind == 1:
        return sqlContext.sql("SELECT Delito, SubDelito,Hora, Fecha, Victima, SubVictima, Edad, Genero, Nacionalidad, Provincia,Canton, LOWER(Distrito) as Distrito from oij")
    else:
        return sqlContext.sql("SELECT LOWER(ProvinciaCantonDistrito) AS ProvinciaCantonDistrito, PoblacionMayor15, TasaNetaParticipacion, TasaOcupacion, TasaDesempleoAbierto, PorcentajeEconomicamenteInactivo, RelacionDependenciaEconomica from inec")

In [ ]:
#Se llama a la función con el número entero correspondiente y se registra el nuevo dataframe
#como la tabla almacenada anteriormente en el sistema, se aplica a ambos conjuntos de datos
oij = minusculas(1)
sqlContext.registerDataFrameAsTable(oij, "oij")
inec = minusculas(2)
sqlContext.registerDataFrameAsTable(inec, "inec")

In [ ]:
'''
Entradas: No posee
Salidas: Un dataframe
Descripción general: Se encarga de hacer la consulta sql para mostrar los datos del OIJ
que no coinciden con ningún dato del INEC
'''
def sacaNoExistentes():
    #Esta consulta se encarga de sacar los distintos por medio del uso de DISTINCT que no se encuentren
    #en la consulta realizada a los datos del INEC
    return sqlContext.sql("SELECT DISTINCT(Distrito) FROM oij WHERE NOT EXISTS(SELECT 1 FROM inec WHERE inec.ProvinciaCantonDistrito = oij.Distrito)") 

In [ ]:
#Llama a la función y lo almacena como un dataframe nuevo y una tabla nueva
noCoinciden = sacaNoExistentes()
sqlContext.registerDataFrameAsTable(noCoinciden, "noCoincidencias")

In [ ]:
#Se muestran los datos que no coincidieron, principalmente fue porque el OIJ utiliza abreviaciones y no usa tildes,
#el INEC no
noCoinciden.show()

In [ ]:
'''
Entradas: No posee
Salidas: Un dataframe
Descripción general: Se encarga de contar los elementos de la tabla de noCoincidencias
estos elementos corresponen a todos aquellos distritos del OIJ que no corresponden a distritos del INEC
'''
def cuentaNoExistentes():
    return sqlContext.sql("SELECT COUNT(*) as num FROM noCoincidencias") 

In [ ]:
#Almacena el dataframe con el dato
numNoCoinciden = cuentaNoExistentes()

In [ ]:
#Se despliega el número correspondiente
numNoCoinciden.show()
print("Cantidad de registros que no coincidieron: ", noCoinciden.count())

In [ ]:
#Se modifican datos del dataframe del INEC para que correspondan a algunos datos del OIJ
#esto se encarga de buscar por columna los datos que tengan almacenado los nombres de la manera indicada
#luego se modifican por sus correpondientes en el dataframe del OIJ
inec = inec.withColumn("ProvinciaCantonDistrito", F.when(F.col("ProvinciaCantonDistrito")=='pococí','pococi').otherwise(F.col("ProvinciaCantonDistrito")))
inec = inec.withColumn("ProvinciaCantonDistrito", F.when(F.col("ProvinciaCantonDistrito")=='la unión','la union').otherwise(F.col("ProvinciaCantonDistrito")))
inec = inec.withColumn("ProvinciaCantonDistrito", F.when(F.col("ProvinciaCantonDistrito")=='belén','belen').otherwise(F.col("ProvinciaCantonDistrito")))
inec = inec.withColumn("ProvinciaCantonDistrito", F.when(F.col("ProvinciaCantonDistrito")=='león cortés castro','leon cortes').otherwise(F.col("ProvinciaCantonDistrito")))
inec = inec.withColumn("ProvinciaCantonDistrito", F.when(F.col("ProvinciaCantonDistrito")=='san josé','san jose').otherwise(F.col("ProvinciaCantonDistrito")))
#se guarda el dataframe modificado como una tabla
sqlContext.registerDataFrameAsTable(inec, "inec")

In [ ]:
#Se integran por medio de un inner join los conjuntos de datos,y se almacenan como un nuevo dataframe
#este dataframe será enviado a postgres
datosIntegrados = sqlContext.sql("SELECT a.Delito, a.SubDelito, a.Hora, a.Fecha, a.Victima, a.SubVictima, a.Edad, a.Genero, a.Nacionalidad, a.Provincia, a.Canton, a.Distrito, b.PoblacionMayor15, b.TasaNetaParticipacion, b.TasaOcupacion, b.TasaDesempleoAbierto, b.PorcentajeEconomicamenteInactivo, b.RelacionDependenciaEconomica FROM oij a INNER JOIN inec b ON a.Distrito = b.ProvinciaCantonDistrito")

In [ ]:
datosIntegrados.printSchema()

In [ ]:
#Se realiza un cast para poder convertir los datos de string a los que originalmente pertenecen
datosIntegrados=datosIntegrados.withColumn('TasaDesempleoAbierto',datosIntegrados['TasaDesempleoAbierto'].cast("float").alias('TasaDesempleoAbierto'))
datosIntegrados=datosIntegrados.withColumn('TasaNetaParticipacion',datosIntegrados['TasaNetaParticipacion'].cast("float").alias('TasaNetaParticipacion'))
datosIntegrados=datosIntegrados.withColumn('TasaOcupacion',datosIntegrados['TasaOcupacion'].cast("float").alias('TasaOcupacion'))
datosIntegrados=datosIntegrados.withColumn('PorcentajeEconomicamenteInactivo',datosIntegrados['PorcentajeEconomicamenteInactivo'].cast("float").alias('PorcentajeEconomicamenteInactivo'))
datosIntegrados=datosIntegrados.withColumn('RelacionDependenciaEconomica',datosIntegrados['RelacionDependenciaEconomica'].cast("float").alias('RelacionDependenciaEconomica'))
datosIntegrados=datosIntegrados.withColumn('Fecha',to_date(unix_timestamp(col('Fecha'), 'yyyy-MM-dd').cast("timestamp")))

In [ ]:
#Se imprime los datos para observar el cambio correctamente
datosIntegrados.printSchema()

In [ ]:
#Se revisa que el cast que se hizo anteriormente no alterara registros
datosIntegrados.show()

In [ ]:
#Se crea una sesión para que spark pueda acceder correctamente a postgres
spark = SparkSession \
    .builder \
    .appName("Basic JDBC pipeline") \
    .config("spark.driver.extraClassPath", "postgresql-42.1.4.jar") \
    .config("spark.executor.extraClassPath", "postgresql-42.1.4.jar") \
    .getOrCreate()

In [ ]:
'''
Entradas: Un dataframe y un string
Salidas: No tiene
Descripción general: Se encarga de que por medio de la sesión anteriormente abierta 
spark pueda acceder correctamente a la sesión de postgres y seguidamente replicar el
dataframe enviado como una tabla propia del postgres
'''
def mandaPostgres(dataframe, nombre):
    dataframe \
        .write \
        .format("jdbc") \
        .mode('overwrite') \
        .option("url", "jdbc:postgresql://localhost/") \
        .option("user", "postgres") \
        .option("password", "password") \
        .option("dbtable", nombre) \
        .save()

In [ ]:
#Se ejecuta la función mandaPostgres para cargar los registros de la tabla datosIntegrados
mandaPostgres(datosIntegrados.select("Delito","SubDelito","Hora","Fecha","Victima","SubVictima","Edad","Genero","Nacionalidad","Provincia","Canton","Distrito","PoblacionMayor15","TasaNetaParticipacion","TasaOcupacion","TasaDesempleoAbierto","PorcentajeEconomicamenteInactivo","RelacionDependenciaEconomica"),"prueba")


# Visualizaciones en Spark

## 1. Cantidad de delitos y la tasa de ocupación para los 10 distritos con más delitos

A continuación se mostrará una comparativa entre los delitos y la tasa de ocupación entre los 10 distritos con más delitos en la cuál se ordenará de mayor a menor la cantidad de delitos para poder determinar el distrito con más delitos y su tasa de ocupación en el año 2011 según el INEC junto al OIJ..
    

In [ ]:
#Se crea un dataframe de datosIntegrados para poder realizar consultas en dicha tabla
sqlContext.registerDataFrameAsTable(datosIntegrados,"datosIntegrados")

'''
Entradas: No tiene
Salidas: Un dataframe
Descripción general: Se encarga de realizar una consulta la cual retorna un
dataframe comparando la cantidad de delitos y la tasa de ocupación com más delitos en el país.
'''
def top10Distritos():
    return sqlContext.sql("SELECT DISTINCT distrito, COUNT(delito) as Cantidad_Delitos, SUM(tasaocupacion)/COUNT(delito) as Tasa_Ocupacion FROM datosIntegrados group by distrito order by 2 DESC LIMIT 10") 
#Se envia la función a una variables
top10Distritos=top10Distritos()
#Seguidamente se muestra el dataframe obtenido
top10Distritos.show()

In [ ]:
#Separa los registros del dataframe anterior a listas para poder graficar y se procede a graficar
y= top10Distritos.toPandas() ["distrito"].values.tolist()
x= top10Distritos.toPandas() ["Cantidad_Delitos"].values.tolist()
z= top10Distritos.toPandas() ["Tasa_Ocupacion"].values.tolist()

#Se crea la figura de la gráfica
fig1, ax1 = plt.subplots()
#Obtenemos una lista con las posiciones
y_pos = np.arange(len(y))
#Creamos la gráfica pasando los valores en el eje X, Y
plt.barh(y_pos[::-1], x, align='center', alpha=1)
#Añadimos la etiqueta de nombre de cada distrito
plt.yticks(y_pos, y[::-1])
#añadimos una etiqueta en el eje X
plt.xlabel('Cantidad Delitos')
#Se reajusta las gráficas para leerlas correctamente
fig1.set_size_inches(10, 5)
#Y una etiqueta superior
plt.title('Top 10 distritos con más Cantidad de Delitos')
plt.show()

#Se crea la figura de la gráfica
fig2, ax2 = plt.subplots()
#Obtenemos una lista con las posiciones
y_pos = np.arange(len(y))
#Creamos la gráfica pasando los valores en el eje X, Y
plt.barh(y_pos[::-1], z, align='center', alpha=1)
#Añadimos la etiqueta de nombre de cada distrito
plt.yticks(y_pos, y[::-1])
#añadimos una etiqueta en el eje X
plt.xlabel('Tasa de Ocupación')
#Se reajusta las gráficas para leerlas correctamente
fig2.set_size_inches(10, 5)
#Y una etiqueta superior
plt.title('Top 10 distritos con más Cantidad de Delitos y con su Tasa de Ocupación')
plt.figure(figsize=(30,30))
plt.show()


## 2. Cantidad de delitos por día de la semana para el distrito con más delitos

Seguidamente se presentará la comparativa con respecto a los delitos del cantón con más delitos en cada día de la semana, así se visualizará que día hubo mayor auge de delitos y que día no hubo menos delitos en el año 2011 según el INEC junto al OIJ.

In [ ]:
'''
Entradas: No tiene
Salidas: Un dataframe
Descripción general: Se encarga de realizar una consulta la cual retorna un
dataframe comparando la cantidad de delitos y con los días de la semana en con respecto
al distrito con más delitos.
'''
def delitosDia():
    return sqlContext.sql("SELECT distrito,COUNT(delito) as Cantidad_Delitos, date_format(Fecha, 'EEEE') as Dia_Semana FROM datosIntegrados WHERE distrito='carmen' group by distrito,Dia_Semana") 
#Se envia la función a una variable
delitosDia=delitosDia()
#Seguidamente se muestra el dataframe obtenido
delitosDia.show()

In [ ]:
#Separa los registros del dataframe anterior a listas para poder graficar y se procede a graficar
x=delitosDia.toPandas() ["Dia_Semana"].values.tolist()

y= delitosDia.toPandas() ["Cantidad_Delitos"].values.tolist()
#Se crea la figura de la gráfica
fig, ax = plt.subplots()
#Colocamos una etiqueta en el eje Y
ax.set_ylabel('Cantidad Delitos')
#Colocamos una etiqueta en el eje X
ax.set_title('Cantidad de delitos por día de la semana para el distrito con más delitos')
#Creamos la gráfica de barras utilizando x,y
plt.bar(x, y)
#Se reajusta las gráficas para leerlas correctamente
fig.set_size_inches(10, 5)
#Finalmente mostramos la gráfica con el método show()
plt.show()

## 3. Cantidad de delitos por tipo y por distrito

En la siguiente gráfica se mostrarán la cantidad de delitos por tipo del distrito seleccionado, en este caso se seleccionó el distrito con más delitos el cual corresponde a el distrito del "carmen" en el año 2011 según el INEC junto al OIJ.

In [ ]:
'''
Entradas: No tiene
Salidas: Un dataframe
Descripción general: Se encarga de realizar una consulta la cual retorna un
dataframe comparando la cantidad de delitos con el tipo correspondiente, seleccionando el
distrito que se quiera, en este caso se seleccionó el distrito del "carmen".
'''
def delitosTipo():
    return sqlContext.sql("SELECT distrito,COUNT(delito) as Cantidad_Delitos, delito as Tipo_Delito FROM datosIntegrados WHERE distrito='carmen' group by distrito,delito ") 
#Se envia la función a una variables
delitosTipo=delitosTipo()
#Seguidamente se muestra el dataframe obtenido
delitosTipo.show()

In [ ]:
#Separa los registros del dataframe anterior a listas para poder graficar y se procede a hacerlo
x=delitosTipo.toPandas() ["Tipo_Delito"].values.tolist()

y= delitosTipo.toPandas() ["Cantidad_Delitos"].values.tolist()
 
fig, ax = plt.subplots()
#Colocamos una etiqueta en el eje Y
ax.set_ylabel('Cantidad Delitos')
#Colocamos una etiqueta en el eje X
ax.set_title('Cantidad de delitos por tipo para el distrito con más delitos')
#Creamos la grafica de barras utilizando x,y
plt.bar(x, y)
#Se reajusta las gráficas para leerlas correctamente
fig.set_size_inches(10, 5)
#Finalmente mostramos la gráfica con el método show()
plt.show()

## 4. Cantidad de delitos por sexo para todo el conjunto de datos

Seguidamente se mostrará la cantidad de delitos por sexo que hay alrededor de toda la cantidad de datos al unir las tablas del INEC con las del OIJ, para saber que sexo es el que comete más delitos en el año 2011 según el INEC junto al OIJ.

In [ ]:
'''
Entradas: No tiene
Salidas: Un dataframe
Descripción general: Se encarga de realizar una consulta la cual retorna un
dataframe comparando la cantidad de delitos con respecto al sexo de todos los datos.
'''
def delitosSexo():
    return sqlContext.sql("SELECT genero, COUNT(delito) as Cantidad_Delitos FROM datosIntegrados group by genero order by Cantidad_Delitos DESC") 
#Se envia la función a una variables
delitosSexo=delitosSexo()
#Seguidamente se muestra el dataframe obtenido
delitosSexo.show()

In [ ]:
#Separa los registros del dataframe anterior a listas para poder graficar y se procede a hacerlo
x=delitosSexo.toPandas() ["genero"].values.tolist()

y= delitosSexo.toPandas() ["Cantidad_Delitos"].values.tolist()
 
fig, ax = plt.subplots()
#Colocamos una etiqueta en el eje Y
ax.set_ylabel('Cantidad Delitos')
#Colocamos una etiqueta en el eje X
ax.set_title('Cantidad de delitos por sexo')
#Creamos la gráfica de barras utilizando x,y
plt.bar(x, y)
#Se reajusta las gráficas para leerlas correctamente
fig.set_size_inches(10, 5)
#Finalmente mostramos la gráfica con el método show()
plt.show()

## 5. Cantidad de delitos con respecto a su subdelito de los 10 distritos con más delitos

Finalmente se mostrará la cantidad de delitos con su respectivo subdelito de los 10 distritos con más delitos del país en el año 2011.

In [ ]:
'''
Entradas: No tiene
Salidas: Un dataframe
Descripción general: Se encarga de realizar una consulta la cual retorna un
dataframe comparando la cantidad de delitos con su respectivo subdelito de los 10 distritos
con más delitos.
'''
def subdelitosCant():
    return sqlContext.sql("SELECT Subdelito, COUNT(delito) as Cantidad_Delitos FROM datosIntegrados group by Subdelito ORDER BY Cantidad_Delitos DESC limit 10") 
#Se envía la función a una variable
subdelitosCant=subdelitosCant()
#Seguidamente se muestra el dataframe obtenido
subdelitosCant.show()

In [ ]:
#Separa los registros del dataframe anterior a listas para poder graficar y se procede a hacerlo
y=subdelitosCant.toPandas() ["Subdelito"].values.tolist()

x= subdelitosCant.toPandas() ["Cantidad_Delitos"].values.tolist()

#Se crea la figura de la gráfica
fig, ax = plt.subplots()
#Obtenemos una lista con las posiciones
y_pos = np.arange(len(y))
#Creamos la gráfica pasando los valores en el eje X, Y
plt.barh(y_pos[::-1], x, align='center', alpha=1)
#Añadimos la etiqueta de nombre de cada distrito
plt.yticks(y_pos, y[::-1])
#añadimos una etiqueta en el eje X
plt.xlabel('Cantidad Delitos')
#Se reajusta las gráficas para leerlas correctamente
fig.set_size_inches(10, 5)
#Y una etiqueta superior
plt.title('Top 10 Tipo de Subdelitos con su respectiva cantidad')
plt.show()

# Conclusiones

1. Se puede concluir por medio de la gráfica número 1 que el distrito en los que se han cometido más delitos a lo largo del 2011 es el "carmen" y a su vez este tiene una tasa de ocupación de un 53.44% aproximadamente, por lo cual con respecto a esta tasa no es la menor de todas, pero aproximadamente el 46% de la población es desempleada, por lo cual puede que sea la razón de que sea un distrito problemático y con más crimenes a lo largo del 2011.



2. Igualmente podemos observar que el día en el que se cometen más cantidad de delitos en el distrito del "carmen" son los sábados, según la gráfica 2, lo cual puede implicar que al ser un día no laboral, lo que son los robos sean más eficaces ya que hay más personas fuera de su casa y muchas de ellas se toman estos días para salir de fiesta o pasear, por lo cual la delincuencia crece, además se puede observar en la gráfica 3 que el tipo de delito  más cometido es el hurto, este mismo es uno de los más sencillos, pero aparentemente más populares según la gráfica.



3. También se puede apreciar que el sexo que tiende a realizar más crimenes según el INEC junto al OIJ es el sexo masculino con más de 25000 delitos a lo largo del año 2011, según el gráfico 4, por otro lado se puede observar que el subdelito con mayor cantidad de delitos es el subdelito de arma de fuego superando los 6000, por esto se puede concluir que la mayoría de personas que cometen algún delito lo realizan con arma de fuego, por lo cual se debe prestar atención cuando se sale de nuestros hogares siempre estando alerta para que no ocurra una fatalidad.

# Referencias 

Apache Spark. (s. f.). Documentation | Apache Spark. Recuperado 20 de diciembre de 2020, de https://spark.apache.org/docs/latest/


Cesar, J. (2019, 9 noviembre). Tutorial: Creación de gráficas en Python usando matplotlib. Facialix. https://blog.facialix.com/tutorial-creacion-de-graficas-en-python-usando-matplotlib/


Hu, J. (2020, 25 junio). Cómo cambiar el tamaño de la figura en Matplotlib. Delft Stack. https://www.delftstack.com/es/howto/matplotlib/how-to-change-the-figure-size-in-matplotlib/


(s. f.). Cómo Eliminar Los Espacios En Blanco En SQL.Estradawebgroup. Recuperado el 22 de diciembre de 2020 de https://estradawebgroup.com/Post/Como-eliminar-los-espacios-en-blanco-en-SQL/1036.

(2017, 23 diciembre). Change column type from string to date in Pyspark. Recuperado el 22 de diciembre de 2020 de https://stackoverflow.com/questions/47953320/change-column-type-from-string-to-date-in-pyspark/47953572


Khan, S. (2016, 28 febrero). How to convert DataFrame columns from string to float/double in PySpark 1.6?. Recuperado el 23 de diciembre de 2020 de https://stackoverflow.com/questions/35684018/how-to-convert-dataframe-columns-from-string-to-float-double-in-pyspark-1-6

(2015, 19 junio). Connecting from Spark/pyspark to PostgreSQL. Recuperado el 24 de diciembre de 2020 de https://stackoverrun.com/es/q/8514701


Vithal, S. (2020, 20 mayo). How to Update Spark DataFrame Column Values using Pyspark?. Recuperado el 22 de diciembre de 2020 de https://dwgeek.com/how-to-update-spark-dataframe-column-values-using-pyspark.html/


(2017, 06 enero). Censo. 2011. Indicadores económicos, según provincia, cantón y distrito. Recuperado el 20 de diciembre de 2020 de http://inec.cr/documento/censo-2011-indicadores-economicos-segun-provincia-canton-y-distrito


(s. f.). Estadísticas Policiales OIJ. Recuperado el 20 de diciembre de 2020 de https://sitiooij.poder-judicial.go.cr/index.php/apertura/transparencia/estadisticas-policiales